# Data Preprocessing

In [2]:
%pip install torch
%pip install torchvision
import torch
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import numpy as np

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#imaget_norm consist of the mean and std of imagenet dataset
imagenet_norm = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

# The following class will be used to preprocess our data
# Every image in our dataset will be resized to 224x224 and converted to tensors
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    imagenet_norm
])




## Loading the dataset

In [4]:
train_data = datasets.ImageFolder(root='./train', transform=transform)
test_data = datasets.ImageFolder(root='./test', transform=transform)
val_data = datasets.ImageFolder(root='./val', transform=transform)


FileNotFoundError: [Errno 2] No such file or directory: './train'

In [ ]:
# Load pretrained model
# feature extractor will be used to extract features from our images
resnet = models.resnet18(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])
feature_extractor.eval() 

In [ ]:
# Function to extract features
def extract_features(dataset):
    
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    features, labels = [], []
    with torch.no_grad():
        for images, y in loader:
            outputs = feature_extractor(images)
            outputs = outputs.view(outputs.size(0), -1)  
            features.append(outputs.numpy())
            labels.append(y.numpy())
    
    X = np.concatenate(features)
    y = np.concatenate(labels)
    return X, y

In [ ]:
# Extract features for all sets
X_train, y_train = extract_features(train_data)
X_val, y_val = extract_features(test_data)
X_test, y_test = extract_features(val_data)

In [ ]:
# Convet the dataset to pandas dataframe
import pandas as pd
X_train_df = pd.DataFrame(X_train)
y_train_df = pd.DataFrame(y_train)
X_val_df = pd.DataFrame(X_val)
y_val_df = pd.DataFrame(y_val)
X_test_df = pd.DataFrame(X_test)
y_test_df = pd.DataFrame(y_test)